In [1]:
#export
"""
This is for support for transpilation of clis to JS
"""
__all__ = ["JsFunc", "toJsFunc", "executeScriptTags"]
from k1lib.cli.init import BaseCli, fastF; import k1lib.cli.init as init
import k1lib.cli as cli; import k1lib, time, json, base64, math, re

In [58]:
#export
settings = k1lib.settings.cli
settings.add("kjs", k1lib.Settings(), "cli.kjs settings")
kjsSett = settings.kjs
kjsSett.add("jsF",   {}, "All ._jsF() (cli to JS) transpile functions, looks like Dict[type -> _jsF(meta, **kwargs) transpile func]")
kjsSett.add("jsonF", {}, "All ._jsonF() JSON-serialization functions, looks like Dict[type -> _jsonF(obj) func]. See utils.deref.json() docs")
kjsSett.add("pyF",   {}, "(future feature) All ._pyF()   (cli to Python) transpile functions, looks like Dict[type -> _pyF  (meta, **kwargs) transpile func]")
kjsSett.add("cppF",  {}, "(future feature) All ._cppF()  (cli to C++)    transpile functions, looks like Dict[type -> _cppF (meta, **kwargs) transpile func]")
kjsSett.add("javaF", {}, "(future feature) All ._javaF() (cli to Java)   transpile functions, looks like Dict[type -> _javaF(meta, **kwargs) transpile func]")
kjsSett.add("sqlF",  {}, "(future feature) All ._sqlF()  (cli to SQL)    transpile functions, looks like Dict[type -> _sqlF (meta, **kwargs) transpile func]")

Settings:      
- jsF   = {}   ​All ._jsF() (cli to JS) transpile functions, looks like Dict[type -> _jsF(meta, **kwargs) transpile func]                            
- jsonF = {}   ​All ._jsonF() JSON-serialization functions, looks like Dict[type -> _jsonF(obj) func]. See utils.deref.json() docs                   
- pyF   = {}   ​(future feature) All ._pyF()   (cli to Python) transpile functions, looks like Dict[type -> _pyF  (meta, **kwargs) transpile func]   
- cppF  = {}   ​(future feature) All ._cppF()  (cli to C++)    transpile functions, looks like Dict[type -> _cppF (meta, **kwargs) transpile func]   
- javaF = {}   ​(future feature) All ._javaF() (cli to Java)   transpile functions, looks like Dict[type -> _javaF(meta, **kwargs) transpile func]   
- sqlF  = {}   ​(future feature) All ._sqlF()  (cli to SQL)    transpile functions, looks like Dict[type -> _sqlF (meta, **kwargs) transpile func]   
               

In [59]:
#export
kjsSett.jsF[str] = lambda meta: (f"", "String")
kjsSett.jsF[int] = lambda meta: (f"", "parseInt")
kjsSett.jsF[float] = lambda meta: (f"", "parseFloat")
kjsSett.jsF[set] = lambda meta: (f"", "new Set")
kjsSett.jsF[list] = lambda meta: (f"", "Array.from")
kjsSett.jsF[tuple] = lambda meta: (f"", "Array.from")
kjsSett.jsF[abs] = lambda meta: (f"", "Math.abs")
kjsSett.jsF[json.loads] = lambda meta: (f"", "JSON.parse")
kjsSett.jsF[json.dumps] = lambda meta: (f"", "JSON.stringify")
kjsSett.jsF[base64.b64decode] = lambda meta: (f"", "atob")
kjsSett.jsF[base64.b64encode] = lambda meta: (f"", "btoa")
def _jst_round(meta, ndigits=None):
    if ndigits is None: return "", f"Math.round"
    else:
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"{fIdx} = ({dataIdx}) => Math.round(({dataIdx})*Math.pow(10, {ndigits})+Number.EPSILON)/Math.pow(10, {ndigits})", fIdx
kjsSett.jsF[round] = _jst_round
import math; kjsSett.jsF[math.floor] = lambda meta: (f"", "Math.floor")
import html
def _jsF_html_escape(meta):
    fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
    return f"{fIdx} = ({dataIdx}) => {dataIdx}.replace(/&/g, '&amp;').replace(/</g, '&lt;').replace(/>/g, '&gt;').replace(/\"/g, '&quot;').replace(/'/g, '&#39;');", fIdx
kjsSett.jsF[html.escape] = _jsF_html_escape
def _jsonF_range(obj):
    if obj.step != 1: return "[" + ", ".join([f"{e}" for e in obj]) + "]"
    end = f".slice({obj.start})" if obj.start != 0 else ""
    return f"[...Array({obj.stop}).keys()]{end}"
kjsSett.jsonF[range] = _jsonF_range
def _jst_resolve(meta):
    fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
    return f"{fIdx} = async ({dataIdx}) => await {dataIdx}", fIdx
kjsSett.jsF[k1lib.resolve] = _jst_resolve
try:
    import requests
    def _jst_requests_get(meta, headers=None):
        headers = headers or {}
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"{fIdx} = async ({dataIdx}) => await fetch({dataIdx}, {{ method: 'GET', headers: {json.dumps(headers)} }})", fIdx
    kjsSett.jsF[requests.get] = _jst_requests_get
    def _jst_requests_post(meta, headers=None, json=None, data=None):
        headers = headers or {}
        if json: headers = {**headers, "Content-Type": "application/json"}
        body = f"JSON.stringify({json.dumps(body)})" if json else json.dumps(data)
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"{fIdx} = async ({dataIdx}) => await fetch({dataIdx}, {{ method: 'POST', headers: {json.dumps(headers)}, body: {body} }})", fIdx
    kjsSett.jsF[requests.post] = _jst_requests_post
except: pass

In [60]:
#export
def v(x): return x if isinstance(x, str) else json.dumps(x)
def vs(xs): return [x if isinstance(x, str) else json.dumps(x) for x in xs]
class vOld(BaseCli):
    def __init__(self, v:str):
        """Specifies a variable should be taken on the js side"""
        super().__init__(); self.v = v
    def __ror__(self, it): return NotImplemented
    def __str__(self): return f"\ue157{self.v}\ue239" # for injection into grep, and other clis that wants to extract this var out of a completely normal string
    def __repr__(self): return f"<kjs.v v={self.v}>"
    @staticmethod
    def getValue(value):
        """If it's an instance of :class:`v`, then return its internal value, else
return the same object passed in. Useful for streamlining building out other clis"""
        return value.v if isinstance(value, v) else value

In [61]:
#export
class Arg:
    def __init__(self, arg):
        if isinstance(arg, str):
            name = arg; type_ = str; default = ""
        elif isinstance(arg, (tuple, list)):
            if len(arg) == 3:
                name, type_, default = arg
            elif len(arg) == 2:
                name, type_ = arg
                if type_ == str: default = ""
                if type_ == int: default = 1
                if type_ == float: default = 1.0
                if type_ == bool: default = True
                if isinstance(type_, (range, k1lib.serve.slider)): default = round((type_.start + type_.stop)/2) # surprisingly no bug here where .step != 1, because <input type="range"> covered us
                if isinstance(type_, list): default = type_[0]
                if isinstance(type_, k1lib.serve.date): default = time.time() | cli.toIso()
            else: raise Exception(f"Argument has to specify 3 values: (name:str, type, default value), or 2 values: (name:str, type)")
        else: raise Exception(f"Can't parse argument `{arg}`. It must be either the argument name alone, or a tuple of (name:str, type, default value)")
        self.idx = cli.init._jsDAuto(); self.name = name; self.type_ = type_; self.default = default; self.arg = arg
    def html(self):
        idx = self.idx; name = self.name; t = self.type_; default = self.default
        if t == str: return f"<input type='text' value='{default}' placeholder='{name}' id='{idx}' style='padding: 4px 4px;'>"
        if t == int or t == float: return f"<input type='number' value='{default}' placeholder='{name}' id='{idx}' style='padding: 4px 4px;'>"
        if t == bool: return f"<input type='checkbox' {'checked' if default else ''} id='{idx}' style='padding: 4px 4px;'>"
        if isinstance(t, (range, k1lib.serve.slider)): return f"<div style='display: flex; flex-direction: row; align-items: center'><input type='range' id='{idx}' min='{t.start}' max='{t.stop}' value='{default}' step='{t.step}' /><span id='{idx}_preview' style='margin-left: 8px'></span></div>"
        if isinstance(t, list):
            if default not in t: raise Exception(f"Dropdown with options {t} does not have the default option {default}")
            s = "".join([f"<option value='{i}' {'selected' if e == default else ''}>{e}</option>" for i, e in enumerate(t)])
            return f"<select id='{idx}'>{s}</select>"
        if isinstance(t, k1lib.serve.date):
            value = f"value='{default}'" if default else ""; minDate = f"min='{t.minDate}'" if t.minDate else ""; maxDate = f"max='{t.maxDate}'" if t.maxDate else ""
            return f'<input id="{idx}" type="datetime-local" {value} {minDate} {maxDate} />'
        raise Exception(f"Don't know type {t} on .html()")
    def value(self): # grab value of interface
        idx = self.idx; name = self.name; t = self.type_; default = self.default
        q = f"document.querySelector('#{self.idx}')"
        if t == str: return f"{q}.value"
        if t == int: return f"parseInt({q}.value || 0)"
        if t == float: return f"parseFloat({q}.value || 0.0)"
        if t == bool: return f"{q}.checked"
        if isinstance(t, range): return f"parseInt({q}.value)"
        if isinstance(t, k1lib.serve.slider): return f"parseFloat({q}.value)"
        if isinstance(t, list): return f"{json.dumps(t)}[{q}.value]"
        if isinstance(t, k1lib.serve.date): return f"{q}.value"
        raise Exception(f"Don't know type {t} on .value()")
    def preview(self):
        idx = self.idx; name = self.name; t = self.type_; default = self.default
        if isinstance(t, range): return f"document.querySelector('#{self.idx}_preview').innerHTML = parseInt(document.querySelector('#{self.idx}').value);"
        if isinstance(t, k1lib.serve.slider):
            delta = 10**(-math.floor(math.log10(t.step))) # figuring out the correct order of magnitude to round to
            return f"document.querySelector('#{self.idx}_preview').innerHTML = Math.round(document.querySelector('#{self.idx}').value*{delta})/{delta};"
        return ""
    def onchange(self, cb:str):
        idx = self.idx; name = self.name; t = self.type_; default = self.default
        q = f"document.querySelector('#{self.idx}')"
        if t == str or t == int or t == float: return f"{q}.oninput = {cb};"
        if t == bool: return f"{q}.onclick = {cb};"
        if isinstance(t, (range, k1lib.serve.slider, list, k1lib.serve.date)): return f"{q}.oninput = {cb};"
        raise Exception(f"Don't know type {t} on .onchange()")
    def copy(self): return Arg(self.arg)

In [73]:
#export
class _JsFuncInterface:
    def __init__(self, jsFunc, mode, args, debounce, delay):
        self.jsFunc = jsFunc; self.mode = mode; self.args = [a.copy() for a in args]; self.debounce = debounce; self.delay = delay
        if delay and delay < debounce: raise Exception(f"delay value ({delay}) is lower than debounce value ({debounce}), which doesn't make sense. Please raise delay, or lower the debounce value")
    def _repr_html_(self):
        pre = init._jsFAuto(); mode = self.mode; args = self.args; debounce = self.debounce*1000
        postprocess = ""
        if mode == "html":
            res = "val"; postprocess = f"""
containerDiv = document.createElement('div'); containerDiv.innerHTML = val;
for (const scriptElem of containerDiv.querySelectorAll('script')) eval(scriptElem.textContent);"""
        elif mode == "pre": res = "`<pre>${val}</pre>`"
        elif mode == "json": res = "`<pre>${JSON.stringify(val)}</pre>`"
        elif mode == "jsone": res = "`<pre>${JSON.stringify(val, null, 2)}</pre>`"
        elif mode == "table": res = """ '<table>' + val.map((v) => "<tr>" + v.map((e) => `<td>${e}</td>`).join("") + "</tr>").join("") + '</table>' """
        else: raise Exception(f"Unknown interface output mode {mode}")
        inps = "".join([f"<div>{a.name}</div><div>{a.html()}</div>" for a in args])
        inps = f"""<div style='display: grid; grid-template-columns: 1fr 3fr; width: 300px; align-items: center'>{inps}</div>"""
        onchanges = "".join([a.onchange(f"{pre}_reload") for a in args])
        values = ", ".join([a.value() for a in args])
        preview = "\n".join([a.preview() for a in args])
        delayCode = f"(async () => {{ while (true) {{ await new Promise(r => setTimeout(r, {self.delay*1000})); await {pre}_reload_core(); }} }})();" if self.delay else ""
        return f"""\
<!-- k1lib.JsFuncInterface -->
{inps}
<pre id="{pre}_error" style="color: red"></pre><div id="{pre}_loading" style="margin-top: 4px">&nbsp;</div><div id="{pre}_result"></div>
<script src="https://k1js.com/dist/amd/latest.js"></script>
<script>
    k1_loadScript = (src) => {{ return new Promise((resolve, reject) => {{ var script = document.createElement('script'); script.src = src; script.onload = resolve; document.head.appendChild(script); }}); }}
    {pre}_loadF = () => {{
        {self.jsFunc.fn}
        window.{self.jsFunc.fIdx} = {self.jsFunc.fIdx}; {pre}_error = document.querySelector("#{pre}_error");
        {pre}_resultDiv = document.querySelector("#{pre}_result");
        {pre}_loadingDiv = document.querySelector("#{pre}_loading"); const runningCoreHandle = [null];
        const {pre}_reload_core = async (e) => {{
            if (runningCoreHandle[0]) {{ runningCoreHandle[0].cancel = true; }}
            const handle = {{"cancel": false}}; runningCoreHandle[0] = handle;
            try {{
                {pre}_error.innerHTML = ""; {pre}_loadingDiv.innerHTML = "(loading...)";
                const val = await {self.jsFunc.fIdx}({values});
                if (!handle.cancel) {{ {pre}_loadingDiv.innerHTML = "&nbsp;"; {pre}_resultDiv.innerHTML = {res}; {postprocess} }}
            }} catch (e) {{ if (!handle.cancel) {pre}_error.innerHTML = e.stack; console.log(e.stack); }}
            runningCoreHandle[0] = null;
        }};
        let {pre}_reload_timeout = null;
        const {pre}_reload = async (e) => {{
            {preview}
            if ({debounce} <= 0) return {pre}_reload_core();
            else {{ clearTimeout({pre}_reload_timeout); {pre}_reload_timeout = setTimeout({pre}_reload_core, {debounce}); }}
        }}
        {onchanges}; {delayCode}; {pre}_reload(null);
    }};
    if (window.k1lib_jsF_envLoaded) {{
        (async () => {{
            while (window.k1lib_jsF_envLoaded !== 2) await new Promise(r => setTimeout(r, 100)); // waits for 100ms when the env is not ready yet
            {pre}_loadF(); }})();
    }} else {{
        window.k1lib_jsF_envLoaded = 1; // some initial env setups
        (async () => {{ // loading up acorn, a JS AST analyzer
            await k1_loadScript("https://cdnjs.cloudflare.com/ajax/libs/acorn/8.11.3/acorn.js");
            await k1_loadScript("https://cdnjs.cloudflare.com/ajax/libs/acorn-walk/8.3.2/walk.js")
        }})();
        async function extractVariableNamesAtDepthOne(funcString) {{ // courtesy of chatgpt-4.5
            while (!window.acorn) await new Promise(r => setTimeout(r, 100)); // loops until acorn is sure to be loaded
            const ast = acorn.parse(funcString, {{ ecmaVersion: 'latest', sourceType: 'script' }}); const declaredVariables = new Set(); let depth = 0;
            const customWalker = {{
                ...acorn.walk.base,
                Function            (node, state, c) {{ if (depth === 1 && node.id)                         declaredVariables.add(node.id.name); depth++; acorn.walk.base.Function            (node, state, c); depth--; }},
                VariableDeclarator  (node, state, c) {{ if (depth === 1)                                    declaredVariables.add(node.  id.name);        acorn.walk.base.VariableDeclarator  (node, state, c);          }},
                AssignmentExpression(node, state, c) {{ if (depth === 1 && node.left.type === 'Identifier') declaredVariables.add(node.left.name);        acorn.walk.base.AssignmentExpression(node, state, c);          }}
            }}; acorn.walk.recursive(ast, null, customWalker); return Array.from(declaredVariables);
        }}; window.k1lib_jsF_captureLocalVariables = extractVariableNamesAtDepthOne;
        // trying multiple loading schemes, because in a live jupyter nb, it's using umd, while in an exported jupyter nb, it's using amd. The madness of JS lol
        (async () => {{ try {{ require(['k1js'], function(k1js) {{ (async () => {{ window.k1js = await k1js; window.k1lib_jsF_envLoaded = 2; {pre}_loadF(); }})(); }}); }} catch (e) {{}} }})();
        (async () => {{ try {{
            const res = await (await fetch("https://k1js.com/dist/umd/latest.js")).text();
            eval(res); window.k1js = await k1js; window.k1lib_jsF_envLoaded = 2; {pre}_loadF();
        }} catch (e) {{}} }})();
    }}
</script>"""
class JsFunc:
    def __init__(self, fn:str, fIdx:str, args, _async):
        """Represents a generated function from :class:`toJsFunc` .
More docs are available at that class.

This is not supposed to be instantiated by the end user. Please
use :class:`toJsFunc` instead. It's here for documentation purposes only"""
        self.fn = fn; self.fIdx = fIdx; self.args = args; self._async = _async
    def __repr__(self):
        fn = self.fn.split('\n') | cli.head(300).all() | cli.join('\n')
        return f"""<JsFunc {self.fIdx}>\n\nGenerated JS function:\n\n{fn}"""
    def _repr_html_(self):
        fn = self.fn.split('\n') | cli.join('\n')
        header = html.escape(f"<JsFunc {self.fIdx}>") + "<br>Generated JS function:"
        try: return f"""{header}<br><br>{k1lib.fmt.js(fn)}"""
        except: return f"<pre style='overflow-x: auto'>{html.escape(repr(self))}</pre>"
    def interface(self, mode="html", debounce:float=0.03, delay:float=None):
        """Creates an interface.

Different output modes:

- html: expected the js function to return html and will display it as-is
- pre: wraps the function result inside a "pre" tag, which makes html honor white spaces and whatnot
- json: json formats the function result in a compact way on 1 line
- jsone: "json expanded", same as above, but in an expanded way on multiple lines
- table: expects function result to be a table, then displays the table nicely

:param mode: see above
:param debounce: if specified, will wait for that many seconds until the user hasn't modified the input UIs.
    If function is not async (meaning it should executes quite fast), then this param will be ignored"""
        return _JsFuncInterface(self, mode, self.args, debounce if self._async else 0, delay)
def moveOut(code:str): # move "k1_moveOutStart - k1_moveOutEnd" blocks to the top
    pattern = r'//k1_moveOutStart(.*?)//k1_moveOutEnd'; matches = re.findall(pattern, code, re.DOTALL); heads = []
    def process_match(match): heads.append(match.group(1).strip()); return ''
    res = re.sub(pattern, process_match, code, flags=re.DOTALL); pre = '\n'.join(heads); return f"//k1_moveOutStart\n{pre}\n//k1_moveOutEnd\n{res}"
class toJsFunc(BaseCli):
    def __init__(self, *args, delay:float=None):
        """Capture cli operations and transpiles all of them to JS, to quickly
build functionalities in JS. Example::

    # creates an instance of class JsFunc that contains the transpiled js code
    jsf = range(10) | deref() | (toJsFunc("term") | head(kjs.v("term")) & head(6))
    # displays a search interface to jupyter notebook
    jsf.interface("json")

So a little more background on how this works. With clis, it can "capture" other
clis to the right of it if it desires. So, ``toJsFunc`` will capture the 2 ``head``s
and transpile them to js. So Python data never really gets passed through those later
heads. Read more about this at :class:`~k1lib.cli.init.BaseCli` and on the main
cli page at https://k1lib.com/latest/cli/index.html

The arguments passed in, namely "term" here, specifies what arguments should the JS
function have. Then you can reference those JS arguments downstream inside any cli,
like ``head(kjs.v("term"))``.

The input ``range(10) | deref()`` actually gets converted into json and injected as
data directly within the function, so it's pretty sweet. It does require the input
to be comprised of relatively simple data structures, with no arbitrary objects.

This is a relatively experimental/novel feature. The transpilation mechanism can
do a lot, but if it sees that something is too complicated, then it will raise an
error and complain, instead of failing silently.

When creating an interface, you can supply them with multiple different types of
UI:

.. pyexec:: html True

    arg1 = "extraText"
    arg2 = ("on", bool)
    arg3 = ("someNum", int)
    arg4 = ("n", range(0, 10, 2))
    arg5 = ("character", ["reimu", "marisa", "cirno"])
    jsF = list(range(100)) | (toJsFunc(arg1, arg2, arg3, arg4, arg5) | head("n") & head("on*10") | aS("[*x, character, someNum**2, len(extraText)*2]"))
    jsF.interface("json")
"""
        super().__init__(capture=True)
        self.args = [Arg(a) for a in args]
    def __ror__(self, it) -> JsFunc:
        header, fn, _async = k1lib.kast.asyncGuard(self.capturedSerial._jsF(("root",))) # used to be (kast, self.args)
        header = "\n".join([f"    {e}" for e in header.split("\n")]); fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return JsFunc(moveOut(f"""
{dataIdx} = {it | cli.deref.js()};
async function {fIdx}({', '.join([a.name for a in self.args])}) {{
{header}
    return {'await ' if _async else ''}{fn}({dataIdx});
}}"""), fIdx, self.args, _async)
    def _jsF(self, meta):
        """Hidden feature, cause this is getting way too complicated and this idea has not been fully fleshed out yet

.. admonition:: Nested toJsFunc()

    Surprisingly and mind bendingly, this can transpile itself, but the use case is rather different.
    Here's a dead simple example::

        jsF = 3 | (toJsFunc() | op()+2 | (toJsFunc("args", "are", "ignored", "here") | aS("x*2")))

"""
        fIdx = cli.init._jsFAuto(); dataIdx = cli.init._jsDAuto(); argIdx = cli.init._jsDAuto(); pre = cli.init._jsDAuto();
        h1,f1,a1 = k1lib.kast.asyncGuard(self.capturedSerial._jsF(meta))
        return f"""//k1_moveOutStart\nlet {dataIdx} = null;\n{h1}\n//k1_moveOutEnd
{fIdx} = ({argIdx}) => {{ // returns html string that will run the function on load
    {dataIdx} = {argIdx};
    return unescape(`<div id='{pre}_div'></div>
        %3Cscript%3E
            setTimeout({'async ' if a1 else ''}() => {{
                console.log("executed");
                document.querySelector('#{pre}_div').innerHTML = {'await ' if a1 else ''}{f1}({dataIdx})
            }}, 100);
        %3C/script%3E`);
}};""", fIdx
class executeScriptTags(BaseCli):
    def __init__(self):
        """In JS-transpiled code, will extract out all script tags and execute them a
few moments after returning the input html. Example::

    "<div>abc</div><script>alert('abc')</script>" | (toJsFunc() | executeScriptTags()) | op().interface()

The output of the cli in JS-transpiled code will be exactly the same as the input, only
the script tags will be extracted and executed. In normal Python mode, does effectively nothing,
and thus is similar to :class:`~k1lib.cli.utils.iden`."""
        pass
    def __ror__(self, it): return it
    def _jsF(self, meta):
        fIdx = cli.init._jsFAuto(); dataIdx = cli.init._jsDAuto()
        return f"""\
{fIdx} = ({dataIdx}) => {{
    const dummyElem = document.createElement("div"); dummyElem.innerHTML = {dataIdx};
    (async () => {{ // execute all script tags a few moments after returning. This feels iffy and might be error-prone, but it's the simplest solution for now
        while (!window.k1lib_jsF_captureLocalVariables) await (new Promise(r => setTimeout(r, 30))); // for some reason, this edge case can happen, so need to delay until that function is available
        await (new Promise(r => setTimeout(r, 30)));
        try {{ for (const script of dummyElem.getElementsByTagName("script")) {{
            const vars = await k1lib_jsF_captureLocalVariables(`function f1() {{ ${{script.innerHTML}} }}`)
            let s = script.innerHTML; for (const v of vars) s += `window.${{v}} = ${{v}};`; eval(s);
        }} }} catch (e) {{ console.log(`Error encountered: ${{e}}. Stack trace: ${{e.stack}}`) }}
    }})(); return {dataIdx};
}}""", fIdx
        pass

In [63]:
arg1 = "extraText"
arg2 = ("on", bool)
arg3 = ("someNum", int)
arg4 = ("n", range(0, 10, 2))
arg5 = ("character", ["reimu", "marisa", "cirno"])
jsF = range(100) | (toJsFunc(arg1, arg2, arg3, arg4, arg5) | cli.head("n") & cli.head("on*10") | cli.aS("[*x, character, someNum**2, len(extraText)*2]"))
jsF.interface("json")

In [64]:
jsF

<JsFunc _jsF_668_1709767798_23>

Generated JS function:

//k1_moveOutStart

//k1_moveOutEnd

_jsD_447_1709767798_56 = [...Array(100).keys()];
async function _jsF_668_1709767798_23(extraText, on, someNum, n, character) {
    _jsF_668_1709767798_19 = (_jsD_447_1709767798_49) => _jsD_447_1709767798_49.head(n, false)
    _jsF_668_1709767798_20 = (_jsD_447_1709767798_51) => _jsD_447_1709767798_51.head(on*10, false)
    _jsF_668_1709767798_18 = (_jsD_447_1709767798_48) => [_jsF_668_1709767798_19(_jsD_447_1709767798_48), _jsF_668_1709767798_20(_jsD_447_1709767798_48)];
    
    _jsF_668_1709767798_22 = (x) => {
        return [...x, character, Math.pow(someNum, 2), (extraText.length*2)];
    }
    _jsF_668_1709767798_17 = (_jsD_447_1709767798_47) => { return _jsF_668_1709767798_22(_jsF_668_1709767798_18(_jsD_447_1709767798_47)); };
    return _jsF_668_1709767798_17(_jsD_447_1709767798_56);
}

In [9]:
jsF = 3 | (toJsFunc() | cli.op()+2 | (toJsFunc("args", "are", "ignored", "here") | cli.aS("x*2")))
jsF.interface()

Perfect!

In [78]:
!../../export.py cli/kjs --upload=True

2024-04-10 18:11:56,606	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-04-10 18:11:56,629	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15853   0   61%   
10236   1   39%   
Found existing installation: k1lib 1.7
Uninstalling k1lib-1.7:
  Successfully uninstalled k1lib-1.7
Looking in indexes: https://pypi.org/simple, http://mint-2.l:3141/
Processing /home/kelvin/repos/labs/k1lib
  Preparing metadata (setup.py) ... done
  Created wheel for k1lib: filename=k1lib-1.7-py3-none-any.whl size=5091768 sha256=e3c1daca80e32c5b4545f74702e3a7b0b4b2724123ced41cad1a0a20a2e9e100
  Stored in directory: /tmp/pip-ephem-wheel-cache-wx1lhuyc/wheels/fb/be/7d/105fb38c560564c6556311fd52f77295b9fec253e4ba7cfd75
Successfully built k1lib


In [55]:
!../../export.py cli/kjs

2024-03-24 06:13:24,441	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-03-24 06:13:24,454	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15780   0   61%   
10191   1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.7
Uninstalling k1lib-1.7:
  Successfully uninstalled k1lib-1.7
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg

In [39]:
!../../export.py cli/kjs --bootstrap=True

2024-03-19 23:34:31,270	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-03-19 23:34:31,294	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- bootstrapping
Current dir: /home/kelvin/repos/labs/k1lib, /home/kelvin/repos/labs/k1lib/k1lib/cli/../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.7
Uninstalling k1lib-1.7:
  Successfully uninstalled k1lib-1.7
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip a